In [21]:
from pymongo import MongoClient
import pprint
import datetime
import pandas as pd
import numpy as np

In [2]:
client = MongoClient('localhost', 27017)
db = client['fraud_study']
collection = db['fraud_data']

In [3]:
collection.count_documents({})

88

In [36]:
risk_range = [-1, .2, .5, .7, 1]
num = 10
risks = ['No Risk','Low Risk','Medium Risk','High Risk']
blank = np.full((num,len(risks)),' ')
df = pd.DataFrame(blank, columns=risks)

In [37]:
df

,No Risk,Low Risk,Medium Risk,High Risk
0,,,,
1,,,,
2,,,,
3,,,,
4,,,,
5,,,,
6,,,,
7,,,,
8,,,,
9,,,,


In [39]:
for i in range(1,5):
    query = (collection
             .find({'probability': {'$gt': risk_range[i-1], '$lte': risk_range[i]}})
             .sort('time', -1)
             .limit(num)
            )
    for j, entry in enumerate(query):
        
        out = ''
        id_ = entry['data']['object_id']
        name = entry['data']['name']
        t = datetime.datetime.fromtimestamp(entry['time'])
        out += (f'ID: {id_}\nNAME: {name[:30]}\nTIME: {t.replace(second=0, microsecond=0)}')
        df.iloc[j,i-1] = out
#         print(
#             risk_range[i-1], 
#             risk_range[i],
#             entry['data']['object_id'],
#             entry['data']['name'],
#             datetime.datetime.fromtimestamp(entry['time']),
#             entry['probability']
#         )
#         print(out)                                                                                                 
#     print()

In [42]:
df.iloc[:4,2]

0    ID: 5602468\nNAME: Team Wench's Privateer Feas...
1    ID: 5176422\nNAME: N Style Steps Into Spring f...
2    ID: 5143070\nNAME: Counterflows Sunday Day Pas...
3    ID: 4979678\nNAME: Audi 1 competition\nTIME: 2...
Name: Medium Risk, dtype: object

In [44]:
df.nunique().values-1

array([9, 9, 9, 7])